**Corpus Name and Selection Reason**

I chose the Gutenberg corpus which contains a collection of various classical literary texts. Since it would be a nice challenge to tokenize old English with its uncommon vocabulary and different genres with varied sentence structures. 
        
I implemented a bottom-up tokenizer using Byte Pair Encoding since there are words in the corpus that are uncommon to me, thus it would be more efficient than creating a top-down tokenizer.
        
Below is a step-by-step breakdown of the design:

In [ ]:
# Bottom-up Tokenizer for Gutenberg Corpus using BPE (Byte Pair Encoding)

# Import required libraries
import nltk
import re
from collections import Counter
from nltk.corpus import gutenberg

# Download the selected corpus if not already available
corpus_name = 'gutenberg'
nltk.download(corpus_name)

**The clean\_corpus() function takes the raw text and splits it into individual tokens using regex**
    
*   Words, contractions, and possessives (e.g., "sister's", "don't", "it's").
*   Numbers (e.g., "1865").
*   Punctuation (e.g., commas, periods, and quotes).
*   After splitting, the tokens are processed to ensure all alphabetic tokens are converted to lowercase

In [2]:
# 2. Clean and tokenize the corpus
def clean_corpus(corpus_raw):
    """
    Tokenize the corpus using regex to match words, contractions, possessives, 
    numbers, punctuation, and hyphenated words. Convert alphabetic tokens to lowercase.

    Args:
        corpus_raw (str): The raw text of the corpus.

    Returns:
        tokens (list): List of cleaned and tokenized words.
    """
    # Regex pattern to match contractions and possessives as one token
    pattern = r"\b\w+(?:[-']\w+)*\b|[^\w\s]"

    # Match words, contractions, possessives, punctuation, and hyphenated words
    tokens = re.findall(pattern, corpus_raw)

    # Lowercase alphabetic tokens or those containing hyphens or apostrophes
    tokens = [token.lower() if any(c.isalpha() for c in token) else token for token in tokens]

    return tokens

**The get\_vocab() function** counts the frequency of each token in the corpus using Python’s Counter class. This allows the BPE algorithm to identify frequently co-occurring tokens for merging.


**The get\_stats() function** identifies frequent adjacent token pairs with the highest frequency in the text, providing input for the next merge step.


**The merge\_pair() function** takes the most frequent adjacent token pair and merges them into a single token. This helps reduce the overall vocabulary size and improves efficiency.

In [3]:
# 3. BPE Helper Functions
def get_vocab(corpus):
    """
    Create a frequency dictionary of tokens from the corpus.

    Args:
        corpus (list): List of tokenized words.

    Returns:
        vocab (Counter): Frequency dictionary of tokens.
    """
    return Counter(corpus)

def get_stats(vocab):
    """
    Calculate the frequency of adjacent token pairs in the vocabulary.

    Args:
        vocab (Counter): Frequency dictionary of tokens.

    Returns:
        pairs (Counter): Frequency dictionary of adjacent token pairs.
    """
    pairs = Counter()
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols) - 1):
            pairs[(symbols[i], symbols[i + 1])] += freq
    return pairs

def merge_pair(pair, vocab):
    """
    Merge the most frequent adjacent token pair in the vocabulary.

    Args:
        pair (tuple): The pair of adjacent tokens to be merged.
        vocab (Counter): Frequency dictionary of tokens.

    Returns:
        merged_vocab (dict): Updated vocabulary with merged token pair.
    """
    merged_vocab = {}
    bigram = ' '.join(pair)
    replacement = ''.join(pair)

    for word in vocab:
        new_word = word.replace(bigram, replacement)
        merged_vocab[new_word] = vocab[word]

    return merged_vocab

**The my_tokenizer function** applies Byte Pair Encoding (BPE) to iteratively merge the most frequent adjacent token pairs in a given corpus, reducing the vocabulary size while preserving token structure.

In [ ]:
# 4. Tokenizer function using BPE
def my_tokenizer(corpus, num_merges):
    """
    Tokenize the corpus using Byte Pair Encoding (BPE) by iteratively merging 
    the most frequent token pairs.

    Args:
        corpus (list): List of tokenized words.
        num_merges (int): Number of merges to perform during BPE.

    Returns:
        list: Final list of tokenized words after BPE merges.
    """
    vocab = get_vocab(corpus)

    for i in range(num_merges):
        pairs = get_stats(vocab)
        if not pairs:
            break

        # Select the most frequent pair to merge
        best_pair = max(pairs, key=pairs.get)
        vocab = merge_pair(best_pair, vocab)

        print(f"Step {i + 1}: Merged {best_pair}")

    # Return the final list of merged tokens
    return list(vocab.keys())

**Main code to run the tokenizer**

In [ ]:
# 5. Main code to run the tokenizer
print("Loading and cleaning the Gutenberg corpus...")

# Load the corpus and clean it
corpus_raw = load_corpus()
tokens = clean_corpus(corpus_raw)

# Set the number of merges for BPE, decided 50 arbitrarily
num_merges = 50

# Tokenize the corpus using BPE
my_tokenized_list = my_tokenizer(tokens, num_merges)

# Display the final list of tokens
print("Tokenization complete. Final tokenized list:")
print(my_tokenized_list)